In [4]:
import os
import praw

In [8]:
reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
    user_agent='ios:python.finetune.reddit:v0.1(finetune with reddit data)'
)

In [33]:
data = []
topic = 'worldnews'

for submission in reddit.subreddit(topic).top(limit=50):
    submission.comments.replace_more(limit=0)
    data.append({
        'prompt':submission.selftext,
        'completion':submission.comments[0].body
    })

In [34]:
len(data)

10